In [1]:
# Visualización Imagen
# ==============================================================================
# nibabel - librería para cargar imágenes nifti
import nibabel as nib 

# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes
import radiomics
from radiomics import featureextractor  
import os

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Preprocesado y análisis
# ==============================================================================
import statsmodels.api as sm
from scipy import stats
from scipy.stats import pearsonr

Vamos a hacer una prueba de la extracción de tan solo unos pocos pacientes:

In [2]:
#Cambiamos al directorio que contiene las imagenes de interes
path = "E:\TFG\BBDD_5pacientes" 
os.chdir(path)

#Estas listas contienen los nombres de los archivos
lista_imagenes = []
lista_etiquetas = []

with os.scandir(path) as files:
  # Iteramos cada uno de los archivos del directorio
    for file in files:
        if file.name.endswith(('_mask.nii.gz')): # COJO TODAS LAS IMÁGENES QUE ACABAN EN MASK.NII.GZ, ES DECIR, LAS MÁSCARAS
          # añadimos las etiquetas a una lista y las imagenes a otra
            lista_etiquetas.append(file.name) # AÑADO LAS MÁSCARAS A UNA LISTA
        else:
            lista_imagenes.append(file.name) # AÑADO LAS IMÁGENES A OTRA LISTA
lista_imagenes = sorted(lista_imagenes)
lista_etiquetas = sorted(lista_etiquetas)
print(os.getcwd())
print(lista_imagenes)
print(lista_etiquetas)

E:\TFG\BBDD_5pacientes
['075_img.nii.gz', '079_img.nii.gz', '080_img.nii.gz', '081_img.nii.gz']
['075_mask.nii.gz', '079_mask.nii.gz', '080_mask.nii.gz', '081_mask.nii.gz']


In [3]:
# ------------------ Extractor de características originales + derivadas ------------------
# Inicializamos el extractor
extractor = featureextractor.RadiomicsFeatureExtractor(r"C:\Users\saral\OneDrive\Escritorio\TFG\Params.yml")
extractor
path = "E:\TFG\BBDD_5pacientes" # Si queremos extraer a todos los pacientes, habrá que cambiar el path
os.chdir(path)
#print(os.getcwd())
caracteristicas = []  # CREO UNA LISTA CON TODAS LAS CARACTERÍSTICAS

In [4]:
# Introducimos las imagenes en el extractor
for img, label in zip(lista_imagenes,lista_etiquetas):
    print(img, label)
    # Obtenemos las características extraidas y las añadimos a una lista 
    # para después generar un DataFrame
    result= extractor.execute(img, label)
    caracteristicas.append(result)

075_img.nii.gz 075_mask.nii.gz


Calculating Local Binary Pattern in 2D, but extracting features in 3D. Use with caution!


079_img.nii.gz 079_mask.nii.gz


Calculating Local Binary Pattern in 2D, but extracting features in 3D. Use with caution!


080_img.nii.gz 080_mask.nii.gz


Calculating Local Binary Pattern in 2D, but extracting features in 3D. Use with caution!


081_img.nii.gz 081_mask.nii.gz


Calculating Local Binary Pattern in 2D, but extracting features in 3D. Use with caution!


In [5]:
# Generamos los dataframes correspondientes
data = pd.DataFrame(caracteristicas, index = lista_imagenes)
data.shape

(4, 1584)

In [6]:
#eliminamos las caracteristicas irrelevantes (versión del software, tamaño de la imagen...)
columns = data.columns
for column in columns:
    if "diagnostics" in column:
        data.drop([column], axis=1, inplace=True)

In [7]:
# Exportacion a excel
os.chdir("E:\TFG\Radiomics")
data.to_excel('Datos_radiomicos.xlsx')

In [8]:
pd.set_option('display.max_rows', 1000) 
pd.set_option('display.max_columns', 1000)
tabla = pd.read_excel("E:\TFG\Radiomics\Datos_radiomicos.xlsx", index_col=0, engine='openpyxl')
print(tabla.shape)
tabla

(4, 1562)


,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,original_shape_Sphericity,original_shape_SurfaceArea,original_shape_SurfaceVolumeRatio,original_shape_VoxelVolume,original_firstorder_10Percentile,original_firstorder_90Percentile,original_firstorder_Energy,original_firstorder_Entropy,original_firstorder_InterquartileRange,original_firstorder_Kurtosis,original_firstorder_Maximum,original_firstorder_MeanAbsoluteDeviation,original_firstorder_Mean,original_firstorder_Median,original_firstorder_Minimum,original_firstorder_Range,original_firstorder_RobustMeanAbsoluteDeviation,original_firstorder_RootMeanSquared,original_firstorder_Skewness,original_firstorder_TotalEnergy,original_firstorder_Uniformity,original_firstorder_Variance,original_glcm_Autocorrelation,original_glcm_JointAverage,original_glcm_ClusterProminence,original_glcm_ClusterShade,original_glcm_ClusterTendency,original_glcm_Contrast,original_glcm_Correlation,original_glcm_DifferenceAverage,original_glcm_DifferenceEntropy,original_glcm_DifferenceVariance,original_glcm_JointEnergy,original_glcm_JointEntropy,original_glcm_Imc1,original_glcm_Imc2,original_glcm_Idm,original_glcm_Idmn,original_glcm_Id,original_glcm_Idn,original_glcm_InverseVariance,original_glcm_MaximumProbability,original_glcm_SumEntropy,original_glcm_SumSquares,original_glrlm_GrayLevelNonUniformity,original_glrlm_GrayLevelNonUniformityNormalized,original_glrlm_GrayLevelVariance,original_glrlm_HighGrayLevelRunEmphasis,original_glrlm_LongRunEmphasis,original_glrlm_LongRunHighGrayLevelEmphasis,original_glrlm_LongRunLowGrayLevelEmphasis,original_glrlm_LowGrayLevelRunEmphasis,original_glrlm_RunEntropy,original_glrlm_RunLengthNonUniformity,original_glrlm_RunLengthNonUniformityNormalized,original_glrlm_RunPercentage,original_glrlm_RunVariance,original_glrlm_ShortRunEmphasis,original_glrlm_ShortRunHighGrayLevelEmphasis,original_glrlm_ShortRunLowGrayLevelEmphasis,original_glszm_GrayLevelNonUniformity,original_glszm_GrayLevelNonUniformityNormalized,original_glszm_GrayLevelVariance,original_glszm_HighGrayLevelZoneEmphasis,original_glszm_LargeAreaEmphasis,original_glszm_LargeAreaHighGrayLevelEmphasis,original_glszm_LargeAreaLowGrayLevelEmphasis,original_glszm_LowGrayLevelZoneEmphasis,original_glszm_SizeZoneNonUniformity,original_glszm_SizeZoneNonUniformityNormalized,original_glszm_SmallAreaEmphasis,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_gldm_DependenceEntropy,original_gldm_DependenceNonUniformity,original_gldm_DependenceNonUniformityNormalized,original_gldm_DependenceVariance,original_gldm_GrayLevelNonUniformity,original_gldm_GrayLevelVariance,original_gldm_HighGrayLevelEmphasis,original_gldm_LargeDependenceEmphasis,original_gldm_LargeDependenceHighGrayLevelEmphasis,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,wavelet-LLH_firstorder_10Percentile,wavelet-LLH_firstorder_90Percentile,wavelet-LLH_firstorder_Energy,wavelet-LLH_firstorder_Entropy,wavelet-LLH_firstorder_InterquartileRange,wavelet-LLH_firstorder_Kurtosis,wavelet-LLH_firstorder_Maximum,wavelet-LLH_firstorder_MeanAbsoluteDeviation,wavelet-LLH_firstorder_Mean,wavelet-LLH_firstorder_Median,wavelet-LLH_firstorder_Minimum,wavelet-LLH_firstorder_Range,wavelet-LLH_firstorder_RobustMeanAbsoluteDeviation,wavelet-LLH_firstorder_RootMeanSquared,wavelet-LLH_firstorder_Skewness,wavelet-LLH_firstorder_TotalEnergy,wavelet-LLH_firstorder_Uniformity,wavelet-LLH_firstorder_Variance,wavelet-LLH_glcm_